***GENERATED CODE FOR bostonhousingapp PIPELINE***

**CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM**

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(inStages, inStagesData, stageId, spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(inStages, inStagesData, stageId, spark, config):
        return inStagesData.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                                        delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


**TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA**

In [ ]:
from pyspark.sql.functions import col, udf, round
from pyspark.ml.feature import Normalizer
from pyspark.sql.types import *
import json
from pyspark.ml.feature import MinMaxScaler
from pyspark.sql.functions import col, udf
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col, when
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def vectorAssemblerTransform(df, param):

    dfReturn = df

    if (type(param) == str):
        outcol = param + "_vector"
        assembler = VectorAssembler(inputCols=[param], outputCol=outcol)
        dfReturn = assembler.transform(dfReturn)
        return dfReturn

    if (type(param) == list):
        vecAssembler = VectorAssembler(inputCols=param, outputCol="features")
        new_df = vecAssembler.transform(df)
        return new_df


def to_array(col):
    def to_array_(v):
        return v.toArray().tolist()
    return udf(to_array_, ArrayType(DoubleType()))(col)


def normalizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']

    dfReturn = dfReturn.fillna({feature: '0.0'})

    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature) \
        .withColumnRenamed("feature_cast", feature)

    outcol = feature + "_normalizer"
    p = transformationData["pNorm"]

    featureVector = feature + "_vector"
    dfReturn = vectorAssemblerTransform(dfReturn, feature)

    normalizer = Normalizer(inputCol=featureVector,
                            outputCol=outcol, p=float(p))
    normalized = normalizer.transform(dfReturn)

    dfReturn = normalized.withColumn("final_col", to_array(normalized[outcol])) \
        .select(normalized.schema.names + [col("final_col")[0]])

    dfReturn = dfReturn.drop(outcol).drop(featureVector)\
        .withColumnRenamed("final_col[0]", outcol)
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[outcol], 2))
    return dfReturn


def minMaxScalarTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']

    dfReturn = dfReturn.fillna({feature: '0.0'})
    lowerbound = float(transformationData["min_max_scalar"].get("min", 0.0))
    upperbound = float(transformationData["min_max_scalar"].get("max", 1.0))
    outcol = feature + "_minmaxscalar"

    featureVector = feature + "_vector"
    dfReturn = vectorAssemblerTransform(dfReturn, feature)

    mmScaleModel = MinMaxScaler(
        inputCol=featureVector, outputCol=outcol, min=lowerbound, max=upperbound)
    scaledata = mmScaleModel.fit(dfReturn).transform(dfReturn)

    dfReturn = scaledata

    dfReturn = dfReturn.withColumn("final_col", to_array(dfReturn[outcol]))\
        .select(dfReturn.schema.names + [col("final_col")[0]])

    dfReturn = dfReturn.drop(outcol).drop(featureVector)\
        .withColumnRenamed("final_col[0]", outcol)
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[outcol], 2))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = normalizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CRIM', 'pNorm': '20.58', 'transformation_label': 'Normalizer'}], 'feature': 'CRIM', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
                                               'count': '244', 'mean': '3.75', 'stddev': '9.43', 'min': '0.00906', 'max': '88.9762', 'missing': '0'}, 'transformation': [{'transformation': 'Normalizer', 'selectedAsDefault': 1}]}, {'feature_label': 'CRIM', 'pNorm': '20.58', 'transformation_label': 'Normalizer'})
        transformationDF = transformationDF.drop('CRIM')
        transformationDF = minMaxScalarTransform(transformationDF, {'transformationsData': [{'feature_label': 'ZN', 'min_max_scalar': {'min': '1.02', 'max': '7.56'}, 'transformation_label': 'Min Max Scalar'}], 'feature': 'ZN', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '244', 'mean': '10.42', 'stddev': '21.68', 'min': '0.0', 'max': '100.0', 'missing': '0'}, 'transformation': [{'transformation': 'Min Max Scalar', 'selectedAsDefault': 1}]}, {'feature_label': 'ZN', 'min_max_scalar': {'min': '1.02', 'max': '7.56'}, 'transformation_label': 'Min Max Scalar'})
        transformationDF = transformationDF.drop('ZN')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


**AUTOML FUNCTIONS**

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    df.columns.intersection(listOfFeatures)
    X = df.drop(label, axis=1).values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': sparkDF.columns}
    return data


**READING DATAFRAME**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

try: 
	bostonhousingappdbfs = DBFSConnector.fetch([], {}, "5ec831a338f3b7a0e925d264", spark, "{'url': '/Demo/BostonTrain.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")

except Exception as ex: 
	logging.error(ex)


**TRANSFORMING DATAFRAME**

In [ ]:
try: 
	bostonhousingappautofe = TransformationMain.run(bostonhousingappdbfs,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "CRIM", "pNorm": "20.58", "transformation_label": "Normalizer"}], "feature": "CRIM", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "244", "mean": "3.75", "stddev": "9.43", "min": "0.00906", "max": "88.9762", "missing": "0"}, "transformation": [{"transformation": "Normalizer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "ZN", "min_max_scalar": {"min": "1.02", "max": "7.56"}, "transformation_label": "Min Max Scalar"}], "feature": "ZN", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "244", "mean": "10.42", "stddev": "21.68", "min": "0.0", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "Min Max Scalar", "selectedAsDefault": 1}]}, {"transformationsData": [{}], "feature": "INDUS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "244", "mean": "11.18", "stddev": "6.76", "min": "0.46", "max": "27.74", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{}], "feature": "CHAS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "244", "mean": "0.07", "stddev": "0.26", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{}], "feature": "NOX", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "244", "mean": "0.56", "stddev": "0.12", "min": "0.385", "max": "0.871", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{}], "feature": "RM", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "244", "mean": "6.24", "stddev": "0.69", "min": "3.561", "max": "8.725", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{}], "feature": "AGE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "244", "mean": "69.1", "stddev": "28.45", "min": "2.9", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{}], "feature": "DIS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "244", "mean": "3.87", "stddev": "2.21", "min": "1.1691", "max": "12.1265", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{}], "feature": "RAD", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "244", "mean": "9.48", "stddev": "8.7", "min": "1.0", "max": "24.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{}], "feature": "TAX", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "244", "mean": "412.32", "stddev": "168.29", "min": "187.0", "max": "711.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{}], "feature": "PTRATIO", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "244", "mean": "18.42", "stddev": "2.09", "min": "13.0", "max": "22.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{}], "feature": "B", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "244", "mean": "353.51", "stddev": "95.49", "min": "0.32", "max": "396.9", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{}], "feature": "LSTAT", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "244", "mean": "12.86", "stddev": "7.3", "min": "1.92", "max": "34.37", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{}], "feature": "MEDV", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "244", "mean": "22.13", "stddev": "9.05", "min": "5.0", "max": "50.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"feature": "CRIM_normalizer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "selected": "True", "transformationsData": [{}], "type": "real", "stats": {"count": "244", "mean": "1.0", "stddev": "0.0", "min": "0.9999999999999998", "max": "1.0000000000000002", "missing": "0"}}, {"feature": "ZN_minmaxscalar_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "selected": "True", "transformationsData": [{}], "type": "real", "stats": {"count": "244", "mean": "1.7", "stddev": "1.42", "min": "1.02", "max": "7.559999999999999", "missing": "0"}}]}))

except Exception as ex: 
	logging.error(ex)


**TRAIN MODEL**

In [ ]:
try: 
	dataAutoML=functionRegression(bostonhousingappautofe, ["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT"], "MEDV")

except Exception as ex: 
	logging.error(ex)


**PREDICT ON TRAINED MODEL**

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

